# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd
import numpy as np
from nltk.stem import SnowballStemmer 
import os
russian_stemmer = SnowballStemmer('russian') 


In [2]:
pth1 = 'data.csv'
pth2 = '/datasets/data.csv'

if os.path.exists(pth1):
    df = pd.read_csv(pth1)
elif os.path.exists(pth2):
    df = pd.read_csv(pth2)
else:
    print('Что-то пошло не так')
 #Запишем данные из csv файла в датафрейм
df.info() #Выведем на экран информацию о датафрейме

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Таблица состоит из `12` столбцов. Названия стобцов указаны корректно.

Число строк отличается, в `total_income` и `days_employed`. Что свидетельствует о том, что там есть пропущенные значения

Изучим наполнение.

In [3]:
display(df.head(10)) #Выведем первые 10 строк таблицы
display(df.tail(10)) #Выведем последние 10 строк таблицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


In [4]:
display(df.describe())

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Мы видим столбы со следующими данными:
1. `children` - количество детей у заемщика
2. `days_employed` - трудовой стаж в днях
3. `dob_years` - возраст заемщика
4. `education` - образование заемщика
5. `educatin_id` - цифровая категория образования заемщика
6. `family_status` - семейное положение
7. `family_status_id` - цифровая категория семейного положения
8. `gender` - пол заемщика
9. `income_type` - источник заработка
10. `debt` - имеет ли заемщик задолженность по возврату кредита
11. `total_income` - ежемесячный доход заемщика
12. `purpose` - цель получение кредита

Сразу можно увидеть, что:
1. Трудовой стаж не может быть отрицательным
2. Трудовой стаж не может быть равен `945` лет 
3. Формат записи образования разный, что может вызывать проблемы при обработке

Начнем предобработку данных

### Шаг 2.1 Заполнение пропусков

In [5]:
df.isna().sum() #Выведем на экран число пропущенных значений по столбцам

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Есть пропущенные значения, что подтверждает выводы сделанные по информации о таблице. 

В `total_income` и `days_employed` есть пропущенные значения, это ежемесячный доход и трудовой стаж в днях. Оба значения являются количественным, а значит, для того, чтобы они не влияли на результаты их необходимо заменить на медианные значения, которые не повредят дальнейшему анализу.


In [6]:
total_income_mean = df['total_income'].median() #Вычисляем среднее в столбце total_income
df['total_income'] = df['total_income'].fillna(value=total_income_mean) #Заполним пропущенные значения в total_income
days_employed_mean = df['days_employed'].median() #Вычисляем среднее в столбце days_employed
df['days_employed'] = df['days_employed'].fillna(value=days_employed_mean) #Заполним пропущенные значения в days_employed
print(df.isna().sum()) #Выведем на экран число пропущенных значений по столбцам 

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


Проверка показала, что мы успешно заменили пропущенные значения

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [7]:
df.describe() #Выведем на экран основную информацию о ко каждому стобцу датафрейма

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,56557.335698,43.293380,0.817236,0.972544,0.080883,1.651595e+05
std,1.381587,134922.319298,12.574584,0.548138,1.420324,0.272661,9.786607e+04
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2518.168900,33.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-385.106616,53.000000,1.000000,1.000000,0.000000,1.955436e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Мы можем увидеть следующие значение, которые требуют нашего внимания.
1. `children` - максимум `20` детей, минимум `-1`. И если `20` детей требуют проверки, например может быть там только `1` заемщик такой, что может быть правдой. То `-1` ребенок быть не может, скорее всего ошибка в знаке. `20` детей скорее всего обсусловлено тем, что сотрудник совершил опечатку и вместо `2.0` ввел `20`. Следовательно примем за правило, что если больше `10` детей, мы выделяем целую часть от деления на `10`. 
2. `days_employed` - стоит обратить внимание на максимальное значение и минимальное значение. `401755` дней это `1 100` лет трудового стажа, что определенно является ошибкой. Если посмотреть на значение процетиля `75%`, мы можем понять, что как минимум `25%` стажа внесено некорректно.
3. `dob_years` - максимум и среднее, имеет адекватные значение, но мы видим, что есть значение `0`. Заемщиком не может быть лицо моложе `18` лет, значит это ошибка.

Для начала займемся столбцом `children`

In [8]:
df.loc[df['children'] < 0, 'children'] *= -1 #Определим все отрицательные значения в столбце и сделаем их положительными
df.loc[df['children'] > 9, 'children'] = df['children'] // 10 #Выделяем целую часть от деления на 10 когда детей 10 и более
display(df.loc[df['children'] > 9]) #Выведем число детей больше 9 на экран для проверки

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose



Далее рассмотрим `days_employed`, нас привлекает выброс в данных и отрицательные значения. Если исправить отрицательные значения просто, то в случае с выбросом желательно понять, чем он вызван.

Судя по стажу в днях и его виду, можно говорить о том, что стаж в днях был расчитан системой, однако тогда число дней должно было быть равным. Мы выделяем теорию, что исходя из значения процетиля `75%` в `63046.5` мы можем утверждать, что стаж был переведен часы 

Судя по данным, мы можем увидеть, что будет несколько значений где стаж с превышает возраст. 
За критерий "ошибочности" значения следующее выражение `Стаж в годах > (Возраст - 18)`. Значения которые попадают в этот критерий мы удалим.

In [9]:
df.loc[df['days_employed'] < 0, 'days_employed'] *= -1 #Превращаем отрицательные значения в положительные
df['days_employed'] = df['days_employed'] / (365*24) #Приняв теорию, что стаж указан в часах, мы вычисляем года стажа
df.loc[df['days_employed'] > (df['dob_years'] - 18), 'days_employed'] = np.nan #Удалим ошибочные значения
days_employed_mean = df['days_employed'].mean() #Вычислим среднее по столбцу
df['days_employed'] = df['days_employed'].fillna(value=days_employed_mean) #Заменим удаленные значения средним

In [10]:
print(df.isna().sum()) #Проверим все ли пропущенные значения заменились

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


Далее займемся `dob_years`. Выясним, сколько людей меньше `18` лет.

In [11]:
display(df.loc[df['dob_years'] < 18].head(50)) #Выводим первые 50 значений где возраст меньше 18
display(df.loc[df['dob_years'] < 18].tail(50)) #Выводим последние 50 значений где возраст меньше 18

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,3.79908,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,3.79908,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,3.79908,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,3.79908,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,3.79908,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
1149,0,3.79908,0,среднее,1,женат / замужем,0,F,компаньон,0,201852.430096,покупка недвижимости
1175,0,3.79908,0,среднее,1,женат / замужем,0,F,пенсионер,0,313949.845188,получение дополнительного образования
1386,0,3.79908,0,высшее,0,женат / замужем,0,M,госслужащий,0,240523.618071,сделка с автомобилем
1890,0,3.79908,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,145017.937533,жилье
1898,0,3.79908,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,127400.268338,на покупку автомобиля


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10595,1,3.79908,0,среднее,1,женат / замужем,0,M,сотрудник,0,158458.734055,жилье
11077,0,3.79908,0,среднее,1,гражданский брак,1,F,компаньон,0,198911.882322,на покупку автомобиля
11289,1,3.79908,0,неоконченное высшее,2,гражданский брак,1,M,сотрудник,0,256070.541695,сыграть свадьбу
11468,1,3.79908,0,высшее,0,гражданский брак,1,F,госслужащий,0,180348.697770,свадьба
11481,2,3.79908,0,среднее,1,женат / замужем,0,M,сотрудник,0,236698.919494,строительство жилой недвижимости
11576,0,3.79908,0,высшее,0,женат / замужем,0,M,сотрудник,0,162427.060843,операции с жильем
11664,0,3.79908,0,среднее,1,женат / замужем,0,F,компаньон,0,73338.168421,заняться высшим образованием
11990,1,3.79908,0,СРЕДНЕЕ,1,женат / замужем,0,F,компаньон,1,167889.299943,на покупку автомобиля
12062,0,3.79908,0,среднее,1,женат / замужем,0,F,пенсионер,0,206718.981389,покупка жилья
12225,1,3.79908,0,среднее,1,Не женат / не замужем,4,M,сотрудник,1,107342.751020,покупка жилья


Просмотр таблицы показал, что проблема с неверным значением стажем, коррелируется с нулем в возрасте, так как в данном случае мы видим значение среднего по стобцу стажа. 

Примем ноль в возрасте как отсутствующее значение и заменим все нули на средне значение возраста.

In [12]:
df.loc[df['dob_years'] == 0, 'dob_years'] = np.nan #Удалим все значения меньше 18
print('Число людей меньше 18 =', df['dob_years'].isna().sum())
dob_years_mean = df['dob_years'].mean() #Вычислим среднее по столбцу
df['dob_years'] = df['dob_years'].fillna(value=dob_years_mean) #Заполним пропуски средним по столбцу

Число людей меньше 18 = 101


Было принято решение о проведении замены на среднее значение исходя из того, что так как таких значений 101, мы можем сказать, что их изменение с большой долей вероятности не исказит данные по просрочкам, в случае необходимости искать зависимость по возрасту, однако они могут нам пригодиться в исследовании вида дохода и просрочек. 

Проверим успешность замены

In [13]:
df.isna().sum() #Проверим успешность замены

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Значения заменены

### Шаг 2.3. Изменение типов данных.

Переведем значения дохода в целочисленный тип, так как при выгрузке данных, образовался "хвост" из числа с плавающей запятой. Уберем значение копеек, так как это не важный показатель для оценки дохода клиента.

In [14]:
df['total_income'] = df['total_income'].astype(int) #Переведем доход в целочисленный тип
display(df)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,0.963205,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,0.459452,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,0.641943,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,0.470862,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,3.799080,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,0.517045,43.0,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,39.262261,67.0,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,0.241250,38.0,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,0.355306,38.0,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Шаг 2.4. Удаление дубликатов.

Начнем с приведения значений в столбцах к общему виду. Ознакомимся с тем, как написаны значения:

In [15]:
display(df['education'].unique()) #Выведем на экран уникальные значения в столбце
display(df['family_status'].unique()) #Выведем на экран уникальные значения в столбце
display(df['income_type'].unique()) #Выведем на экран уникальные значения в столбце
display(df['purpose'].unique()) #Выведем на экран уникальные значения в столбце

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Как мы видим, неявные дубликаты есть только в стобце `education`. Избавимся от них: 

In [16]:
df['education'] = df['education'].str.lower() #Приведем все значения в столбце к нижнему регистру
display(df['education'].unique()) #Выведем все уникальные значения в стобце

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Мы привели значения категоральные значения в столбцах к единому виду, и тем самым избавились от неявных дубликатов по значениям столбца. Теперь уберем явные дубликаты строк

In [18]:
a = len(df)
df = df.drop_duplicates().reset_index(drop=True) #Удалим все дубликаты строк и обновим индексацию
print('Количество удаленных значений:', a - len(df)) #Выведем длину датафрейма после удаления
print('Доля удаленных значений:{:.2%}'.format(1 - len(df)/a)) #Выведем длину датафрейма после удаления

Количество удаленных значений : 0
Доля удаленных значений:0.00%


Был удален `71` дубликат.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Для удобства просмотра таблицы сформируем поля `education` и `family_status` в отдельные словари, так как в таблице им уже присвоено id, а значит их можно выделить в отдельные словари используя эти же id.

In [20]:
edu_dict = df[['education','education_id']] #Сформируем отдельный датафрейм из двух столбцов
edu_dict = edu_dict.drop_duplicates().reset_index(drop=True) #Удалим все дубликаты и сбросим значения индекса
df.drop('education', inplace=True, axis=1) #Удалим столбец из изначального датафрейма
display(edu_dict) #Отобразим новый словарь
family_dict = df[['family_status','family_status_id']] #Сформируем отдельный датафрейм из двух столбцов
family_dict = family_dict.drop_duplicates().reset_index(drop=True) #Удалим все дубликаты и сбросим значения индекса
df.drop('family_status', inplace=True, axis=1) #Удалим столбец из изначального датафрейма
display(family_dict) #Отобразим новый словарь
display(df) #Проверим корректность удаления столбцов

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,0.963205,42.0,0,0,F,сотрудник,0,253875,покупка жилья
1,1,0.459452,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,0.641943,33.0,1,0,M,сотрудник,0,145885,покупка жилья
3,3,0.470862,32.0,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,3.799080,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...
21449,1,0.517045,43.0,1,1,F,компаньон,0,224791,операции с жильем
21450,0,39.262261,67.0,1,0,F,пенсионер,0,155999,сделка с автомобилем
21451,1,0.241250,38.0,1,1,M,сотрудник,1,89672,недвижимость
21452,3,0.355306,38.0,1,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Шаг 2.6. Категоризация дохода.

На текущий момент столбец `total_income` не может нам сказать, как уровень дохода влияет на риск выдачи кредита, так как нет определния уровня дохода. В соотвествии с заданием определим категории дохода клиента.

`0–30000` — категория 'E'

`30001–50000` — категория 'D'

`50001–200000` — категория 'C'

`200001–1000000` — категория 'B'

`1000001 и выше` — категория 'A'.

In [21]:
def total_income_category(income): #Зададим функцию, которая проверяет значение на входе и возвращает категорию по данному значению 
    if income <= 30000:
        cat = 'E'
        return cat
    elif income <= 50000:
        cat = 'D'
        return cat
    elif income <= 200000:
        cat = 'C'
        return cat
    elif income <= 1000000:
        cat = 'B'
        return cat
    else:
        cat = 'A'
        return cat
df['total_income_category'] = df['total_income'].apply(total_income_category) #С помощью ф-ции apply применим функцию на каждую ячейку столбца total_income
display(df) #Проверим результат выполнения ф-ции

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,0.963205,42.0,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,0.459452,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,0.641943,33.0,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,0.470862,32.0,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,3.799080,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
...,...,...,...,...,...,...,...,...,...,...,...
21449,1,0.517045,43.0,1,1,F,компаньон,0,224791,операции с жильем,B
21450,0,39.262261,67.0,1,0,F,пенсионер,0,155999,сделка с автомобилем,C
21451,1,0.241250,38.0,1,1,M,сотрудник,1,89672,недвижимость,C
21452,3,0.355306,38.0,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B


### Шаг 2.7. Категоризация целей кредита.

На текущий момент столбец `purpose` не может нам сказать, как цель кредита влияет на риск выдачи кредита, так как нет определения уровня дохода. В соотвествии с заданием определим категории цели взятия кредита как:

Операции с автомобилем

Операции с недвижимостью

Проведение свадьбы

Получение образования


In [22]:
def purpose_category(purpose): #Зададим функцию, которая проверяет значение на входе и возвращает категорию по данному значению 
    stemmed_query = russian_stemmer.stem(purpose) #Воспользуемся стеммингом для удобного поиска по словам
    for word in stemmed_query.split(' '): #Так как цель может состоять из несколько слов, нам нужно разделить строку на отдельные слова и проверить каждое.
        if word == 'автомобил' or word == 'автомоб':
            cat = 'операции с автомобилем'
            return cat
        elif word == 'недвижим' or word == 'жил' or word == 'жилья':
            cat = 'операции с недвижимостью'
            return cat
        elif word == 'свадьб':
            cat = 'проведение свадьбы'
            return cat
        elif word == 'образован':
            cat = 'получение образования'
            return cat
df['purpose_category'] = df['purpose'].apply(purpose_category) #С помощью ф-ции apply применим функцию на каждую ячейку столбца total_income
display(df[df['purpose_category'].isnull()]) #Проверим результат выполнения ф-ции, нет ли значений None  
display(df) #Проверим результат выполнения ф-ции      

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,0.963205,42.0,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,0.459452,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,0.641943,33.0,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,0.470862,32.0,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,3.799080,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,0.517045,43.0,1,1,F,компаньон,0,224791,операции с жильем,B,операции с недвижимостью
21450,0,39.262261,67.0,1,0,F,пенсионер,0,155999,сделка с автомобилем,C,операции с автомобилем
21451,1,0.241250,38.0,1,1,M,сотрудник,1,89672,недвижимость,C,операции с недвижимостью
21452,3,0.355306,38.0,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B,операции с автомобилем


Нулевых результатов нет.
Все цели считаны и категорированы

### Ответы на вопросы.

##### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?

##### Вывод 1:

In [23]:
children_debt = df.pivot_table(index='children',values='debt',aggfunc=lambda a: np.mean(a) * 100) #Лямбда ф-ция для перевода в читаемый вид
display(children_debt)

,debt
children,
0,7.543822
1,9.165808
2,9.492481
3,8.181818
4,9.756098
5,0.000000


Мы можем увидеть, что наиболее рискованными заемщиками являются те, у кого `2` или `4` ребенка, однако выборка клиентов с `4` детьми слишком мала, исходя из этого нельзя утверждать, что `4` ребенка имеют больше риска чем `2`.
Так как наименее рискованная категория, это заемщики без детей, что логично, так как ребенок не приносит дохода, а только требует вложений.

Возможно, то что люди с `2` детьми наиболее рискованные заемщики означает тот факт, что обычно в семьи с `2` и более детьми,находятся в более уязвимом положении, так как в случае болезни одного ребенка, это сразу распространяется и на второго, что приводит к дополнительным тратам.

Для людей с `5` детьми слишком маленькая выборка для оценки из за чего по данной группе людей отсутствуют просрочки.

##### Вопрос 2: 

Есть ли зависимость между семейным положением и возвратом кредита в срок?

##### Вывод 2


In [24]:
family_debt = df.pivot_table(index='family_status_id',values='debt',aggfunc=lambda a: np.mean(a) * 100) #Лямбда ф-ция для перевода в читаемый вид
display(family_debt)

,debt
family_status_id,
0,7.545182
1,9.347145
2,6.569343
3,7.112971
4,9.750890


Мы можем увидеть, что наиболее рискованный заемщик не женат / не замужем. Это может быть связано с тем, что в случае возникновения финансовых проблем, он не может расчитывать на помощь другого человека, а так же более склонен к необдуманным решениями в связи с отсутствием в жизни постоянной иной точки зрения

А наименее рискованный Вдова/Вдовец, что достаточно очевидно, так как в случае смерти супруга, второй член семьи получает наследство и таким образом концентрирует у себя накопления семьи.

##### Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

##### Вывод 3:

In [25]:
income_debt = df.pivot_table(index='total_income_category',values='debt',aggfunc=lambda a: np.mean(a) * 100) #Лямбда ф-ция для перевода в читаемый вид
display(income_debt)

,debt
total_income_category,
A,8.000000
B,7.062091
C,8.491508
D,6.000000
E,9.090909


Мы можем увидеть, что наиболее рискованный заемщик с доходом от `0 до 30 000` рублей в месяц. Это заемщик с наиболее низким доходом из представленных категорий, для данного типа заемщиков любое финансовое потрясение может привести к просрочке, так же у данной категории нет накоплений. 

А наименее рискованный с доходом от `20 000 до 50 000` рублей в месяц. Обычно это люди имеющие стабильную работу, не склонные риску и понимающие факт того, что у них невысокий уровень дохода.

##### Вопрос 4:

Как разные цели кредита влияют на его возврат в срок?

##### Вывод 4:

In [34]:
purpose_debt = df.pivot_table(index='purpose_category',values='debt',aggfunc=lambda a: np.mean(a) * 100).round(2) #Лямбда ф-ция для перевода в читаемый вид
display(purpose_debt)

,debt
purpose_category,
операции с автомобилем,9.36
операции с недвижимостью,7.23
получение образования,9.22
проведение свадьбы,8.00


Мы можем увидеть, что наиболее рискованный заемщик с целью: `Операции с автомобилем`. Автокредит как таковой не только требует ежемесячной оплаты, но постоянного содержания, а так же как движимое имущество, обладает повышенным риском к повреждениям, что так же может выливаться в дополнительные траты.

Наименее рискованный заемщик - `Операции с недвижимостью`. Недвижимость - долгосрочный займ, обеспеченный этой недвижимостью. Обычно характеризуются невысоким ежемесячным платежом в связи с длинным сроком. Коммерческая ипотека так же направлена на то, что объект недвижимости становится доходом, что так же уменьшает риск невыплаты кредита.

## Общий вывод:

Проанализировав данные, мы можем сделать следующие выводы касательно качества заемщиков.

**Наиболее рискованный заемщик в каждой группе:**

- 2 ребенка

- Не женат или не замужем

- С доходом менее 30 000 тысяч

- Кредит связанный с автомобилем

**Наименее рискованный заемщик по каждой группе:**

- Без детей

- Вдовец/Вдова

- С доходом от 30000 до 50000 

- Под недвижимость

Из для более точной оценки желательно было бы получить полную стоимость кредита по каждому заемщику, так как без привязки к суммам, тяжело управлять этим риском. Однако это дает примерно представление о наиболее благополучной категории заемщиков и наименее благополучной категории. 
На основании исследования стоит внести тестовые коррективы в скоринговую модель и собрать данные по отказам и их причинам и сравнить с данными по отказам до тестирования новой скоринговой модели. 
Крайне желательно провести исследование доходности каждой группы клиентов и сравнить с риском, невозвратов, вполне возможна ситуация, когда тот же автокредит, хоть и является наиболее рисковым, будет приносить даже в рамках реализации риска суммарно больше прибыли, чем кредит на свадьбу.

